# Project Group - 5

Members: Ahmad Nabil Maulana, Daan Michel, Gijs Aben, Philine Cremers

Student numbers: 5943442, 4684559 ,    4713656   ,     5036534

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

### Main research question

**What is the influence of a US-state's weath, measured in GDP per capita, on the numer of air traffic movements in that specific state?**

### Sub questions

**How does state GDP per capita relate to the proportion of domestic and international flight movements within the United States, and are there differences in this relationship among states?\
\
Did the impact of the COVID-19 pandemic on air traffic movements in the United States vary significantly among states with different levels of GDP per capita?\
\
Were there pivotal moments or significant turning points in the relationship between state GDP per capita and air traffic movements in the United States, and what factors contributed to these shifts?**



# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: Ahmad Nabil Maulana : Creating the repository, Sub-Question 1 Code and Explanation, Sub-Question 2 Code and Explanation

**Author 2**: Daan Michel

**Author 3**: Philine Cremers : data analysis, visualisation sub question 2, sub question 3

**Author 4**: Gijs Aben : Searching and editing data sets, formulating sub questions, writing the report and analysing results

# Background, Context, and Hypotheses

For this project, analisis of the relationship between a US-State's GDP per capita and its air traffic movements are being conducted. GDP measures the monetary value of final goods and services produced within a state's borders during a specific period, such as a quarter or a year. We aim to determine whether states with higher GDPs exhibit greater air traffic mobility compared to those with lower GDPs, or if there is no discernible correlation.

For the data of the GPD several databases were used. Firstly a dataset from the Bureau of Economic Analysis was used to obtain the GDP per state over the years from 2005 up until 2022. This data is combined with the population data found at Census Bureau. Both these datasets are used to obtain the value of GDP per capita. This value is needed because in this way you can compare the states with eachother. Also, within these datasets the chained 2012 dollars are given. In this way we can account for the inflation and other infleunces and therefore can easily compare 2008 to 2020 for example.

The definition of flight movemets are all air traffic movements within a state. This means that both departing and arriving aircraft are counted. Within the data a distiction of domestic and international flights can be made. This will also be used to answer certain sub-questions.

Before focussing on the US, GDP data from all over the world was considered. However when analising data from the Open-Sky network it was concluded that using this database was quite complicated and that the amount of data was to much to handle. After doing some online research detailed data from the US was found and the main reseach objective was reshaped to the current version.

The fight data that was used was found on the website of the Department of Transportation. As said above the data shows the which carrier performed how many flights from one destination to another in each quarter. During this project only air traffic movements in a state will be considered and not the specific locations of the cities. Important about this data is that either departure, arrival, or both must be within the borders of the US. Futhermore, private aircraft are not included in this dataset and the data ranges from 2005 up until 2022.

# Hypotheses

The expected outcome of this research is that a US-state's wealth and the number of air traffic movements have a positive correlation.

Furthermore there is an expected positive corrilation between the relative number of international flights and the GDP per capita.

Regarding the Covid sub-question it is expected that during this specific time a state's wealth and the number of air traffic movements are not correlated in any way.

Lastly besides the COVID-19 pandemic we have no specific pivotal moments that we expect to find, however we are nontheless interested if there are any.

In [4]:
# this can be changed
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import os
import requests
import io
import zipfile
import pickle
import statsmodels.api as sm


# **Part I - Data Import**

First, we're going to import and combine dataframes from the two datas that we will be using:

* State's GDP data from the Bureau of Economic Analysis - https://www.bea.gov/data/gdp/gdp-state 
* State's population data from Census Bureau - https://www.census.gov/data/tables/time-series/dec/popchange-data-text.html
* State's air traffic mobility data from the Department of Transportation - https://www.transtats.bts.gov/Tables.asp?QO_VQ=EEE&QO_anzr=Nv4%FDPn44vr4%FDf6n6v56vp5%FD

# GDP per Capita Data Import

In [5]:
# import the GDP Data for all the states in the USA
file_path_GDP_per_capita = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/GDP%20%26%20Population/GDP%20per%20Capita.csv'
df_GDP_per_capita = pd.read_csv(file_path_GDP_per_capita)


df_GDP_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 74 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   GeoName  51 non-null     object 
 1   Unit     51 non-null     object 
 2   2005:Q1  51 non-null     float64
 3   2005:Q2  51 non-null     float64
 4   2005:Q3  51 non-null     float64
 5   2005:Q4  51 non-null     float64
 6   2006:Q1  51 non-null     float64
 7   2006:Q2  51 non-null     float64
 8   2006:Q3  51 non-null     float64
 9   2006:Q4  51 non-null     float64
 10  2007:Q1  51 non-null     float64
 11  2007:Q2  51 non-null     float64
 12  2007:Q3  51 non-null     float64
 13  2007:Q4  51 non-null     float64
 14  2008:Q1  51 non-null     float64
 15  2008:Q2  51 non-null     float64
 16  2008:Q3  51 non-null     float64
 17  2008:Q4  51 non-null     float64
 18  2009:Q1  51 non-null     float64
 19  2009:Q2  51 non-null     float64
 20  2009:Q3  51 non-null     float64
 21  2009:Q4  51 non-nu

# The USA Flight Data Import

### A. USA INTERNATIONAL FLIGHT

In [6]:
# Load CSV files
df1 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2005.csv'
df2 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2006.csv'
df3 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2007.csv'
df4 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2008.csv'
df5 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2009.csv'
df6 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2010.csv'
df7 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2011.csv'
df8 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2012.csv'
df9 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2013.csv'
df10 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2014.csv'
df11 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2015.csv'
df12 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2016.csv'
df13 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2017.csv'
df14 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2018.csv'
df15 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2019.csv'
df16 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2020.csv'
df17 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2021.csv'
df18 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/International%20data/T_T100I_SEGMENT_ALL_CARRIER_INTERNATIONAL_2022.csv'

csv_files = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18]

# Initialize an empty list to store dataframes
dfs_international = []

# Load and append each CSV file from URLs to the list of dataframes
for url in csv_files:
    df = pd.read_csv(url)
    dfs_international.append(df)

# Concatenate the dataframes vertically
merged_df_international = pd.concat(dfs_international, ignore_index=True)

# Save the merged dataframe to a new Pickle File
# merged_df.to_pickle('MERGED_DATA_INTERNATIONAL.pkl')



# dfw = pd.read_pickle('/Users/daanmichel/TU Delft/TIL Programming/big-project/Data/Flight data/MERGED_DATA_INTERNATIONAL.pkl')
# print(dfw.info())

In [7]:
merged_df_international.info()
merged_df_international.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426827 entries, 0 to 1426826
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   DEPARTURES_PERFORMED  1426827 non-null  float64
 1   PAYLOAD               1426827 non-null  float64
 2   SEATS                 1426827 non-null  float64
 3   PASSENGERS            1426827 non-null  float64
 4   UNIQUE_CARRIER        1424494 non-null  object 
 5   UNIQUE_CARRIER_NAME   1426633 non-null  object 
 6   REGION                1426633 non-null  object 
 7   CARRIER               1424688 non-null  object 
 8   CARRIER_NAME          1426633 non-null  object 
 9   ORIGIN_AIRPORT_ID     1426827 non-null  int64  
 10  ORIGIN                1426827 non-null  object 
 11  ORIGIN_CITY_NAME      1426827 non-null  object 
 12  ORIGIN_COUNTRY_NAME   1426827 non-null  object 
 13  DEST_AIRPORT_ID       1426827 non-null  int64  
 14  DEST                  1426827 non-

,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_COUNTRY_NAME,AIRCRAFT_TYPE,YEAR,QUARTER
0,0.0,0.0,0.0,0.0,US,US Airways Inc.,A,US,US Airways Inc.,12966,LGW,"London, United Kingdom",United Kingdom,14100,PHL,"Philadelphia, PA",United States,625,2005,1
1,0.0,0.0,0.0,0.0,YV,Mesa Airlines Inc.,D,YV,Mesa Airlines Inc.,14100,PHL,"Philadelphia, PA",United States,16271,YYZ,"Toronto, Canada",Canada,675,2005,1
2,0.0,0.0,0.0,0.0,YV,Mesa Airlines Inc.,D,YV,Mesa Airlines Inc.,16271,YYZ,"Toronto, Canada",Canada,14100,PHL,"Philadelphia, PA",United States,675,2005,1
3,1.0,0.0,0.0,0.0,4N,Air North,I,4N,Air North,10754,BRW,"Barrow, AK",United States,16254,YXY,"Whitehorse, Canada",Canada,407,2005,3
4,1.0,242.0,64.0,1.0,W8,Cargojet Airways Ltd.,I,W8,Cargojet Airways Ltd.,12953,LGA,"New York, NY",United States,16163,YQB,"Quebec, Canada",Canada,715,2005,2


In [8]:
# As the state of either the departure or the destination are not given in a seperate column
# we want to split that from the column of either Origin_City_Name or Dest_City_Name.


def get_state(location):
    parts = location.split(', ')
    if len(parts) == 2:
        city, area = parts
        if area in ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']:
            return area  # It's a US state
        else:
            return 'Foreign'  # It's a foreign city
    else:
        return 'Unknown'  # Invalid entry


merged_df_international['ORIGIN_STATE'] = merged_df_international['ORIGIN_CITY_NAME'].apply(get_state)
merged_df_international['DEST_STATE'] = merged_df_international['DEST_CITY_NAME'].apply(get_state)


In [9]:
merged_df_international.head(15)

,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,...,ORIGIN_COUNTRY_NAME,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_COUNTRY_NAME,AIRCRAFT_TYPE,YEAR,QUARTER,ORIGIN_STATE,DEST_STATE
0,0.0,0.0,0.0,0.0,US,US Airways Inc.,A,US,US Airways Inc.,12966,...,United Kingdom,14100,PHL,"Philadelphia, PA",United States,625,2005,1,Foreign,PA
1,0.0,0.0,0.0,0.0,YV,Mesa Airlines Inc.,D,YV,Mesa Airlines Inc.,14100,...,United States,16271,YYZ,"Toronto, Canada",Canada,675,2005,1,PA,Foreign
2,0.0,0.0,0.0,0.0,YV,Mesa Airlines Inc.,D,YV,Mesa Airlines Inc.,16271,...,Canada,14100,PHL,"Philadelphia, PA",United States,675,2005,1,Foreign,PA
3,1.0,0.0,0.0,0.0,4N,Air North,I,4N,Air North,10754,...,United States,16254,YXY,"Whitehorse, Canada",Canada,407,2005,3,AK,Foreign
4,1.0,242.0,64.0,1.0,W8,Cargojet Airways Ltd.,I,W8,Cargojet Airways Ltd.,12953,...,United States,16163,YQB,"Quebec, Canada",Canada,715,2005,2,NY,Foreign
5,1.0,242.0,64.0,1.0,W8,Cargojet Airways Ltd.,I,W8,Cargojet Airways Ltd.,16271,...,Canada,10540,BED,"Bedford, MA",United States,715,2005,3,Foreign,MA
6,1.0,484.0,62.0,2.0,W8,Cargojet Airways Ltd.,I,W8,Cargojet Airways Ltd.,12953,...,United States,16271,YYZ,"Toronto, Canada",Canada,715,2005,4,NY,Foreign
7,1.0,484.0,62.0,2.0,W8,Cargojet Airways Ltd.,I,W8,Cargojet Airways Ltd.,14112,...,United States,16271,YYZ,"Toronto, Canada",Canada,715,2005,4,FL,Foreign
8,1.0,600.0,3.0,0.0,KAH,Kenmore Air Harbor,D,KAH,Kenmore Air Harbor,13002,...,United States,16278,YZT,"Port Hardy, Canada",Canada,30,2005,3,WA,Foreign
9,1.0,600.0,3.0,0.0,KAH,Kenmore Air Harbor,D,KAH,Kenmore Air Harbor,16260,...,Canada,13002,LKE,"Seattle, WA",United States,30,2005,3,Foreign,WA


### B. USA DOMESTIC FLIGHT

In [10]:
# Do not run this cell

# Load CSV files
df_2005 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2005.csv'
df_2006 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2006.csv'
df_2007 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2007.csv'
df_2008 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2008.csv'
df_2009 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2009.csv'
df_2010 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2010.csv'
df_2011 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2011.csv'
df_2012 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2012.csv'
df_2013 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2013.csv'
df_2014 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2014.csv'
df_2015 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2015.csv'
df_2016 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2016.csv'
df_2017 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2017.csv'
df_2018 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2018.csv'
df_2019 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2019.csv'
df_2020 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2020.csv'
df_2021 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2021.csv'
df_2022 = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/Flight%20data/Domestic%20data/T_T100D_SEGMENT_ALL_CARRIER_DOMESTIC_2022.csv'

csv_files = [df_2005, df_2006, df_2007, 
             df_2008, df_2009, df_2010, 
             df_2011, df_2012, df_2013,
             df_2014, df_2015, df_2016,
             df_2017, df_2018, df_2019,
             df_2020, df_2021, df_2022
             ]

# Initialize an empty list to store dataframes
dfs_domestic = []

# Load and append each CSV file from URLs to the list of dataframes
for url in csv_files:
    df = pd.read_csv(url)
    dfs_domestic.append(df)

# Concatenate the dataframes vertically
merged_df_domestic = pd.concat(dfs_domestic, ignore_index=True)

# Save the merged dataframe to a new Pickle File
# merged_df.to_pickle('MERGED_DATA_DOMESTIC.pkl')


# dfw = pd.read_pickle('/Users/daanmichel/TU Delft/TIL Programming/big-project/Data/Flight data/MERGED_DATA_DOMESTIC.pkl')
# print(dfw.info())

In [11]:
merged_df_domestic.info()
merged_df_domestic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6542868 entries, 0 to 6542867
Data columns (total 20 columns):
 #   Column                Dtype  
---  ------                -----  
 0   DEPARTURES_PERFORMED  float64
 1   PAYLOAD               float64
 2   SEATS                 float64
 3   PASSENGERS            float64
 4   UNIQUE_CARRIER        object 
 5   UNIQUE_CARRIER_NAME   object 
 6   REGION                object 
 7   CARRIER               object 
 8   CARRIER_NAME          object 
 9   ORIGIN_AIRPORT_ID     int64  
 10  ORIGIN                object 
 11  ORIGIN_CITY_NAME      object 
 12  ORIGIN_STATE_NM       object 
 13  DEST_AIRPORT_ID       int64  
 14  DEST                  object 
 15  DEST_CITY_NAME        object 
 16  DEST_STATE_NM         object 
 17  AIRCRAFT_TYPE         int64  
 18  YEAR                  int64  
 19  QUARTER               int64  
dtypes: float64(4), int64(5), object(11)
memory usage: 998.4+ MB


,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,AIRCRAFT_TYPE,YEAR,QUARTER
0,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12184,HOM,"Homer, AK",Alaska,12649,KEB,"Nanwalek, AK",Alaska,35,2005,4
1,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12184,HOM,"Homer, AK",Alaska,14088,PGM,"Port Graham, AK",Alaska,35,2005,4
2,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12184,HOM,"Homer, AK",Alaska,14088,PGM,"Port Graham, AK",Alaska,35,2005,4
3,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12649,KEB,"Nanwalek, AK",Alaska,12184,HOM,"Homer, AK",Alaska,35,2005,4
4,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12649,KEB,"Nanwalek, AK",Alaska,14942,SOV,"Seldovia, AK",Alaska,35,2005,4


In [12]:
# It is convenient if wwe want to merge the data later on to create a column that contains
# a state's origin or destination's abbreviation. The function is given below.

def state_name_to_abbreviation(state_name):
    # Dictionary mapping full state names to their abbreviations
    state_abbreviations = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
        'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
        'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
        'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
        'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
        'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
        'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
        'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
    }
    
    # Convert the state name to its abbreviation using the dictionary
    abbreviation = state_abbreviations.get(state_name, state_name)
    
    return abbreviation

merged_df_domestic['ORIGIN_STATE'] = merged_df_domestic['ORIGIN_STATE_NM'].apply(state_name_to_abbreviation)
merged_df_domestic['DEST_STATE'] = merged_df_domestic['DEST_STATE_NM'].apply(state_name_to_abbreviation)

In [13]:
merged_df_domestic.head(15)

,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,...,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,AIRCRAFT_TYPE,YEAR,QUARTER,ORIGIN_STATE,DEST_STATE
0,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12184,...,Alaska,12649,KEB,"Nanwalek, AK",Alaska,35,2005,4,AK,AK
1,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12184,...,Alaska,14088,PGM,"Port Graham, AK",Alaska,35,2005,4,AK,AK
2,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12184,...,Alaska,14088,PGM,"Port Graham, AK",Alaska,35,2005,4,AK,AK
3,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12649,...,Alaska,12184,HOM,"Homer, AK",Alaska,35,2005,4,AK,AK
4,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,12649,...,Alaska,14942,SOV,"Seldovia, AK",Alaska,35,2005,4,AK,AK
5,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,14088,...,Alaska,12649,KEB,"Nanwalek, AK",Alaska,35,2005,4,AK,AK
6,0.0,0.0,0.0,0.0,2E,Smokey Bay Air Inc.,D,2E,Smokey Bay Air Inc.,14942,...,Alaska,14088,PGM,"Port Graham, AK",Alaska,35,2005,4,AK,AK
7,0.0,0.0,0.0,0.0,2U,Valley Air Express Inc.,D,2U,Valley Air Express Inc.,15570,...,California,12211,HSH,"Las Vegas, NV",Nevada,194,2005,1,CA,NV
8,0.0,0.0,0.0,0.0,2U,Valley Air Express Inc.,D,2U,Valley Air Express Inc.,15570,...,California,12211,HSH,"Las Vegas, NV",Nevada,194,2005,1,CA,NV
9,0.0,0.0,0.0,0.0,2U,Valley Air Express Inc.,D,2U,Valley Air Express Inc.,15570,...,California,12211,HSH,"Las Vegas, NV",Nevada,194,2005,1,CA,NV


In [14]:
# Both datasets have some entries where the column flights performed is 0.0
# these have to be dropped

merged_df_domestic = merged_df_domestic[merged_df_domestic['DEPARTURES_PERFORMED'] != 0.0]
merged_df_international = merged_df_international[merged_df_international['DEPARTURES_PERFORMED'] != 0.0]

merged_df_international.head()
merged_df_domestic.head()

,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,...,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,AIRCRAFT_TYPE,YEAR,QUARTER,ORIGIN_STATE,DEST_STATE
528,1.0,0.0,0.0,0.0,HB,Homer Air,D,HB,Homer Air,12649,...,Alaska,10016,A03,"Hallo Bay, AK",Alaska,35,2005,3,AK,AK
529,1.0,0.0,0.0,0.0,NC,Northern Air Cargo Inc.,D,NC,Northern Air Cargo Inc.,11445,...,Alaska,10170,ADQ,"Kodiak, AK",Alaska,216,2005,3,AK,AK
530,1.0,0.0,0.0,0.0,UA,United Air Lines Inc.,D,UA,United Air Lines Inc.,12173,...,Hawaii,14771,SFO,"San Francisco, CA",California,627,2005,3,HI,CA
531,1.0,0.0,9.0,0.0,8D,"Dynamic Aviation, Inc. d/b/a Beacon",D,8D,Servant Air Inc.,13863,...,Alaska,10243,AKK,"Akhiok, AK",Alaska,194,2005,1,AK,AK
532,1.0,0.0,9.0,1.0,8D,"Dynamic Aviation, Inc. d/b/a Beacon",D,8D,Servant Air Inc.,12772,...,Alaska,13934,ORI,"Port Lions, AK",Alaska,194,2005,1,AK,AK


# **Part II - Data Processing**

## Sub question 1:
- ## How does state GDP per capita relate to the proportion of domestic and international flight movements within the United States, and are there differences in this relationship among states?

To anser this first sub question all the states GDPs per capita are visualized in a diagram. The purpose of this is to get an understandig of the division of the GDP per capita in the United States.

In [15]:
# Three final datasets that now have the correct configuration and 
# can be used in the analysis further on

merged_df_international.info()
merged_df_domestic.info()
df_GDP_per_capita.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422891 entries, 3 to 1426826
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   DEPARTURES_PERFORMED  1422891 non-null  float64
 1   PAYLOAD               1422891 non-null  float64
 2   SEATS                 1422891 non-null  float64
 3   PASSENGERS            1422891 non-null  float64
 4   UNIQUE_CARRIER        1420558 non-null  object 
 5   UNIQUE_CARRIER_NAME   1422697 non-null  object 
 6   REGION                1422697 non-null  object 
 7   CARRIER               1420752 non-null  object 
 8   CARRIER_NAME          1422697 non-null  object 
 9   ORIGIN_AIRPORT_ID     1422891 non-null  int64  
 10  ORIGIN                1422891 non-null  object 
 11  ORIGIN_CITY_NAME      1422891 non-null  object 
 12  ORIGIN_COUNTRY_NAME   1422891 non-null  object 
 13  DEST_AIRPORT_ID       1422891 non-null  int64  
 14  DEST                  1422891 non-

,GeoName,Unit,2005:Q1,2005:Q2,2005:Q3,2005:Q4,2006:Q1,2006:Q2,2006:Q3,2006:Q4,...,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3,2022:Q4
0,Alabama,Millions of chained 2012 dollars,0.041061,0.041516,0.041406,0.041851,0.041946,0.042160,0.042149,0.042187,...,0.040644,0.040683,0.041200,0.041769,0.041803,0.042400,0.042352,0.042257,0.042383,0.042795
1,Alaska,Millions of chained 2012 dollars,0.072059,0.073017,0.072578,0.073650,0.075521,0.078017,0.078935,0.081374,...,0.068424,0.069635,0.069118,0.069142,0.069453,0.069736,0.066909,0.066763,0.068175,0.068861
2,Arizona,Millions of chained 2012 dollars,0.049998,0.050848,0.051771,0.051900,0.052868,0.052995,0.053617,0.054278,...,0.046381,0.047288,0.047495,0.048233,0.048760,0.049965,0.049683,0.049439,0.049915,0.050315
3,Arkansas,Millions of chained 2012 dollars,0.039161,0.039377,0.039602,0.040429,0.040444,0.040930,0.040591,0.040317,...,0.039640,0.039744,0.040465,0.040848,0.041068,0.041453,0.042106,0.041783,0.041922,0.042253
4,California,Millions of chained 2012 dollars,0.055989,0.056502,0.057283,0.057772,0.059016,0.058816,0.059138,0.059504,...,0.068249,0.069460,0.070883,0.072297,0.073217,0.074434,0.072598,0.072506,0.073194,0.073635


In [21]:
import pandas as pd
import plotly.express as px

# Assuming df_GDP_per_capita is the DataFrame containing the GDP data

# Filter out the 'GeoName' column and the columns for GDP from 2005 and 2022
df_GDP_2005 = df_GDP_per_capita[['GeoName'] + [col for col in df_GDP_per_capita.columns if col.startswith('2005')]]
df_GDP_2022 = df_GDP_per_capita[['GeoName'] + [col for col in df_GDP_per_capita.columns if col.startswith('2022')]]

# Melt the data for easy plotting
df_GDP_melted_2005 = pd.melt(df_GDP_2005, id_vars='GeoName', var_name='Quarter', value_name='GDP per capita')
df_GDP_melted_2022 = pd.melt(df_GDP_2022, id_vars='GeoName', var_name='Quarter', value_name='GDP per capita')

# Create an interactive bar chart for 2005 using Plotly Express
fig_2005 = px.bar(df_GDP_melted_2005, x='GeoName', y='GDP per capita', color='Quarter',
                  title='U.S. State GDP per Capita in 2005',
                  labels={'GDP per capita': 'GDP per Capita', 'Quarter': 'Quarter'})
fig_2005.update_layout(xaxis_title='State', yaxis_title='GDP per capita (chained 2012 dollars)')

# Create an interactive bar chart for 2022 using Plotly Express
fig_2022 = px.bar(df_GDP_melted_2022, x='GeoName', y='GDP per capita', color='Quarter',
                  title='U.S. State GDP per Capita in 2022',
                  labels={'GDP per capita': 'GDP per Capita', 'Quarter': 'Quarter'})
fig_2022.update_layout(xaxis_title='State', yaxis_title='GDP per capita (chained 2012 dollars)')

fig_2005.show()
fig_2022.show()

From both barplots above, we can see that in both 2005 and 2022, the District of Columbia has the highest GDP per capita, while Mississippi is the state with the lowest GDP per capita in the USA. 

Additionally, we aim to examine the historical trend of GDP per capita by state to identify whether there are any significant increases or drops in specific years. We will visualise the historical trend by creating a line plot for the GDP of all the states from 2005-2022.

In [20]:
import pandas as pd
import plotly.express as px

# Ensure the 'GeoName' column and the columns for GDP from '2005:Q1' to '2022:Q4' are selected
# Assuming the columns are labeled from '2005:Q1' to '2022:Q4' in the df_GDP_per_capita DataFrame
df_GDP_1 = df_GDP_per_capita[['GeoName'] + [col for col in df_GDP_per_capita.columns if '2005:Q1' <= col <= '2022:Q4']]

# Melt the data for easy plotting
df_GDP_melted = pd.melt(df_GDP_1, id_vars='GeoName', var_name='Quarter', value_name='GDP per capita')

# Extract the year from the 'Quarter' column
df_GDP_melted['Year'] = df_GDP_melted['Quarter'].str.extract('(\d{4})')

# Aggregate the GDP per capita data by year and state to get annual GDP per capita
df_GDP_annual = df_GDP_melted.groupby(['GeoName', 'Year'])['GDP per capita'].mean().reset_index()

# Create an interactive line chart using Plotly Express to show the historical trend per year
fig = px.line(df_GDP_annual, x='Year', y='GDP per capita', color='GeoName',
              title='Historical Trend of U.S. State GDP per Capita (2005-2022) Per Year',
              labels={'GDP per capita': 'GDP per Capita', 'Year': 'Year'})

fig.update_layout(xaxis_title='Year', yaxis_title='GDP per capita (chained 2012 dollars)')
fig.show()


Based on the historical trend depicted above, it is evident that the GDP per capita of most states has exhibited a small but consistent upward trajectory year by year. However, what stands out most is the significant decline at the close of 2019 and the beginning of 2020. This period coincides with the onset of the COVID-19 pandemic, leading us to conclude that the pandemic had a substantial impact on the economic state of the USA during 2019-2020.

### GDP per capita - Domestic Flights

In the next part we are going to research if there is a correlation between a state's GDP per capita and the number of domestic flight from all the states.

In [22]:
# 1a. Count flights for each origin state and year-quarter
origin_flight_counts = merged_df_domestic.groupby(['ORIGIN_STATE_NM', 'YEAR', 'QUARTER']).size().reset_index(name='Flight_Count')

# 1b. Count flights for each destination state and year-quarter
dest_flight_counts = merged_df_domestic.groupby(['DEST_STATE_NM', 'YEAR', 'QUARTER']).size().reset_index(name='Flight_Count')

# Rename columns for merge
origin_flight_counts = origin_flight_counts.rename(columns={'ORIGIN_STATE_NM': 'State'})
dest_flight_counts = dest_flight_counts.rename(columns={'DEST_STATE_NM': 'State'})

# 2. Reshape GDP dataframe to long format
gdp_melted = df_GDP_per_capita.melt(id_vars=['GeoName'], value_vars=df_GDP_per_capita.columns[4:], var_name='Year:Quarter', value_name='GDP')

# Split 'Year:Quarter' column into separate 'YEAR' and 'QUARTER' columns
gdp_melted['YEAR'] = gdp_melted['Year:Quarter'].str.split(':', expand=True)[0].astype(int)
gdp_melted['QUARTER'] = gdp_melted['Year:Quarter'].str.extract('Q(\d)')[0].astype(int)
gdp_melted.drop('Year:Quarter', axis=1, inplace=True)

# 3. Merge flight_counts with gdp_melted for both origin and destination states
origin_merged = pd.merge(origin_flight_counts, gdp_melted, left_on=['State', 'YEAR', 'QUARTER'], right_on=['GeoName', 'YEAR', 'QUARTER'], how='inner')
dest_merged = pd.merge(dest_flight_counts, gdp_melted, left_on=['State', 'YEAR', 'QUARTER'], right_on=['GeoName', 'YEAR', 'QUARTER'], how='inner')

# 4. Compute correlation for each state
origin_correlations = origin_merged.groupby('GeoName').apply(lambda group: group['Flight_Count'].corr(group['GDP'])).reset_index(name='Origin_Correlation')
dest_correlations = dest_merged.groupby('GeoName').apply(lambda group: group['Flight_Count'].corr(group['GDP'])).reset_index(name='Dest_Correlation')

# Combine the correlations
combined_correlations = pd.merge(origin_correlations, dest_correlations, on='GeoName', how='inner')
combined_correlations

,GeoName,Origin_Correlation,Dest_Correlation
0,Alabama,0.351224,0.338837
1,Alaska,0.568305,0.570872
2,Arizona,0.049436,0.048286
3,Arkansas,0.263573,0.208091
4,California,0.869821,0.862484
5,Colorado,0.631813,0.619792
6,Connecticut,0.554655,0.557535
7,Delaware,0.229023,0.213773
8,Florida,0.312831,0.305253
9,Georgia,-0.026097,-0.034853


In [23]:
states = combined_correlations['GeoName']
origin_corr = combined_correlations['Origin_Correlation']
dest_corr = combined_correlations['Dest_Correlation']

# Create an interactive bar plot
fig = go.Figure()

# Add bars for origin correlation
fig.add_trace(go.Bar(x=states, y=origin_corr, name='Origin Correlation', marker_color='blue'))

# Add bars for destination correlation
fig.add_trace(go.Bar(x=states, y=dest_corr, name='Destination Correlation', marker_color='red'))

# Customize the layout
fig.update_layout(title='Correlations for Origin and Destination States in USA Domestic Flights',
                  xaxis=dict(title='States'),
                  yaxis=dict(title='Correlation'),
                  barmode='group')  # 'group' will place bars side-by-side

# Show the plot
fig.show()


The similarity between the origin and destination correlations for each state suggests consistency in the relationship between GDP per capita and flight counts, whether considering flights originating from or destined to a particular state. This consistency is a positive sign and could indicate the data's reliability.

From the graph above, we can see that the magnitude and direction of the correlations vary across states. Some states show positive correlations, indicating that as GDP per capita increases, flight counts also increase. Conversely, other states show negative correlations, suggesting the opposite trend. This indicates that factors influencing flight counts may differ from state to state.

There are states (like California and Illinois) where the correlation values are relatively high (either positively or negatively). This suggests that in these states, there's a strong relationship between GDP per capita and flight counts. Some states have correlations close to zero, indicating that there's little to no linear relationship between GDP per capita and flight counts for those states. This is for example the case in Georgia home of Atlanta International the biggest airport in passenger numbers in the world, the airport is a tranfer hub for passeger between the east and west coast. This leads to a large amount of domestic flights not correlated to the state's GDP per capita. States known for robust economic activity and commercial hubs (like California) show strong positive correlations, which aligns with expectations. However, while correlation indicates a relationship, it does not imply causation.

For states where the correlation is negative, this could suggest two potential interpretations:
1. If the flight counts decrease when GDP per capita rises, it could suggest that businesses and individuals in the state might be using alternative means of communication (like video conferencing) or transportation (like rail or road) during prosperous times. This might be because they have the resources to invest in technology or choose other travel means that might be perceived as more convenient or efficient.
2.  If flight counts increase when GDP decreases, it might mean that there's a surge in travel, possibly due to more affordable ticket prices, promotions by airlines, increased business travels in search of opportunities, or other factors.

For states where the correlation is positive, this could suggest that the economies rely on air transport. Economic growth might lead to better infrastructure, including airport facilities, and more investment from airlines, leading to an increase in flight services.

Each state has its own economic, geographic, and social dynamics that can influence the relationship between GDP per capita and flight counts. While a negative correlation might suggest a reduced reliance on air transport for some states, it is not the case for all.

To see more detailed relationship between both GDP per capita and domestic flights a regression analysis is performed below:

In [24]:

total_dom_flight_counts = pd.concat([origin_flight_counts, dest_flight_counts], ignore_index=True)
total_dom_flight_counts = total_dom_flight_counts.groupby(['State', 'YEAR', 'QUARTER'])['Flight_Count'].sum().reset_index()


# Merge total flight counts with GDP data
merged_data = pd.merge(total_dom_flight_counts, gdp_melted, left_on=['State', 'YEAR', 'QUARTER'], right_on=['GeoName', 'YEAR', 'QUARTER'], how='inner')

# Convert GDP to numeric and handle missing values
merged_data['GDP'] = pd.to_numeric(merged_data['GDP'], errors='coerce')
merged_data = merged_data.dropna(subset=['GDP', 'Flight_Count'])

# Prepare data for regression
X = sm.add_constant(merged_data['GDP'])  # Adding a constant for the intercept
y = merged_data['Flight_Count']

# Run regression
model = sm.OLS(y, X).fit()

# Show summary of regression
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Flight_Count   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     561.1
Date:                Fri, 10 Nov 2023   Prob (F-statistic):          3.60e-115
Time:                        15:56:43   Log-Likelihood:                -34567.
No. Observations:                3500   AIC:                         6.914e+04
Df Residuals:                    3498   BIC:                         6.915e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5932.9579    409.914    -14.474      0.000   -6736.652   -5129.264
GDP         1.781e+05   7518.074     23.687      0.000    1.63e+05    1.93e+05
==============================================================================
Omnibus:                     2311.700   Durbin-Watson:                   0.063
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35538.024
Skew:                           2.949   Prob(JB):                         0.00
Kurtosis:                      17.454   Cond. No.                         94.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-squared value is 0.138, which means that approximately 13.8% of the variability in the number of flights is explained by the GDP per capita. This is a relatively low value, suggesting that GDP per capita alone doesn't explain much of the variability in flight counts.

The amount of departure flights per state is mapped below using bubble plot:

In [25]:
# merged_df_domestic.head()
# merged_df_international.head()

In [24]:

# 1. Count flights for each origin state and year-quarter
origin_dom_flight_counts = merged_df_domestic.groupby(['ORIGIN_STATE', 'YEAR', 'QUARTER']).size().reset_index(name='Number of Flights')

# Assuming df is your flight data DataFrame and is already loaded
# Grouping the data by "ORIGIN_STATE" and counting the number of flights

# Ensure 'Year' and 'Quarter' columns are in string format
origin_dom_flight_counts['YEAR'] = origin_dom_flight_counts['YEAR'].astype(str)
origin_dom_flight_counts['QUARTER'] = origin_dom_flight_counts['QUARTER'].astype(str)
origin_dom_flight_counts['Year_Quarter'] = origin_dom_flight_counts['YEAR'] + ' Q' + origin_dom_flight_counts['QUARTER']

# Grouping the data by state and year-quarter
state_flight_counts = (
    origin_dom_flight_counts[origin_dom_flight_counts['ORIGIN_STATE'].notnull()]
    .groupby(['ORIGIN_STATE', 'Year_Quarter'])
    .size()
    .reset_index(name='Number of Flights')
)

# Adding latitude and longitude coordinates for each state
state_coords = {
    'AL': (32.806671, -86.791130),
    'AK': (61.370716, -152.404419),
    'AZ': (33.729759, -111.431221),
    'AR': (34.969704, -92.373123),
    'CA': (36.116203, -119.681564),
    'CO': (39.059811, -105.311104),
    'CT': (41.597782, -72.755371),
    'DE': (39.318523, -75.507141),
    'FL': (27.994402, -81.760254),
    'GA': (33.040619, -83.643074),
    'HI': (21.094318, -157.498337),
    'ID': (44.240459, -114.478828),
    'IL': (40.349457, -88.986137),
    'IN': (39.849426, -86.258278),
    'IA': (42.011539, -93.210526),
    'KS': (38.526600, -96.726486),
    'KY': (37.668140, -84.670067),
    'LA': (31.169546, -91.867805),
    'ME': (44.693947, -69.381927),
    'MD': (39.063946, -76.802101),
    'MA': (42.230171, -71.530106),
    'MI': (43.326618, -84.536095),
    'MN': (45.694454, -93.900192),
    'MS': (32.741646, -89.678696),
    'MO': (38.456085, -92.288368),
    'MT': (46.921925, -110.454353),
    'NE': (41.125370, -98.268082),
    'NV': (38.313515, -117.055374),
    'NH': (43.452492, -71.563896),
    'NJ': (40.298904, -74.521011),
    'NM': (34.840515, -106.248482),
    'NY': (42.165726, -74.948051),
    'NC': (35.630066, -79.806419),
    'ND': (47.528912, -99.784012),
    'OH': (40.388783, -82.764915),
    'OK': (35.565342, -96.928917),
    'OR': (44.572021, -122.070938),
    'PA': (40.590752, -77.209755),
    'RI': (41.680893, -71.511780),
    'SC': (33.856892, -80.945007),
    'SD': (44.299783, -99.438828),
    'TN': (35.747845, -86.692345),
    'TX': (31.054487, -97.563461),
    'UT': (40.150032, -111.862434),
    'VT': (44.045876, -72.710686),
    'VA': (37.769337, -78.169968),
    'WA': (47.400902, -121.490494),
    'WV': (38.491226, -80.954453),
    'WI': (44.268543, -89.616508),
    'WY': (42.755966, -107.302490),
    'DC': (38.897438, -77.026817),
    'PR': (18.220833, -66.590149),
    'VI': (18.335765, -64.896335),
    'MP': (15.097900, 145.673900),
    'AS': (-14.271000, -170.132000),
    'GU': (13.444300, 144.793700)
}
# Create a DataFrame for state information
state_info = (
    pd.DataFrame(state_coords.items(), columns=['State', 'Coords'])
    .assign(**{'lat': lambda df: df['Coords'].str[0], 'lon': lambda df: df['Coords'].str[1]})
    .drop(columns='Coords')
)

# Merge flight counts with state information
state_dom_flight_counts = pd.merge(origin_dom_flight_counts, state_info, left_on='ORIGIN_STATE', right_on='State')

# Creating the bubble map
fig = px.scatter_geo(
    state_dom_flight_counts,
    lon               = 'lon',
    lat               = 'lat',
    size              = 'Number of Flights',
    color             = 'Number of Flights',
    hover_name        = 'State',
    hover_data        = {'State': False, 'Number of Flights': True},
    color_continuous_scale = 'Viridis',
    size_max          = 50,
    locationmode      = 'USA-states',
    title             = 'Number of Domestic Flights Originating from Each State Over Time',
    scope             = 'usa',
    template          = 'plotly',
    animation_frame   = 'Year_Quarter'
)

# Layout Updates
fig.update_geos(
    landcolor          = 'rgb(210, 180, 140)',  # Light brown color for land
    showocean          = True, oceancolor = 'rgb(135, 206, 250)',  # Light blue color for ocean
    showlakes          = True, lakecolor  = 'rgb(173, 216, 230)',  # Lighter blue color for lakes
    showland           = True,
    showsubunits       = True, subunitcolor = 'rgb(255, 255, 255)',
    showcountries      = False,
    showcoastlines     = True, coastlinecolor = 'rgb(255, 0, 0)',  # Red color for coastline
    showrivers         = True, rivercolor = 'rgb(70, 130, 180)'    # Steel blue color for rivers
)



fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0},
                  coloraxis_colorbar=dict(title='Number of Flights'))

fig.show()


From the map above, it is apparent that Alaska with a relatively high GDP per capita stands out with an exorbitant high number of domestic flight departures in comparison to other states with comparable GDP per capita. This phenomenon can be attributed to several factors, primarily stemming from Alaska’s unique geographical location and topography. Following Alaska there are three states that stand out in high amount of domestic flights, being also the three biggest states in population, California, Texas and Florida. To get a better understanding of these outcomes we go a bit more into the characteristics of these states. 

Alaska is situated at a considerable distance from the contiguous United States, separated by vast expanses of sea and rugged terrain. This geographical isolation means that road or rail travel to and from Alaska is not as feasible or efficient as it is in other parts of the United States. As a result, air travel becomes a more practical and sometimes the only viable option for transportation, both for residents and visitors, leading to a higher volume of flight departures.

California has the biggest population of all the states by quite a margin. Besides that, California has a relitively high GDP per capita and as shown in the correlation diagram does have a high correlation between GDP per capita and flight movements. This is in lesser amount the case for Texas and almost not at all for Florida, which could indicate that the number of people living in each state has a bigger impact on the amount of flights. However besides a large population Florida also has a lot of popular tourist destinations that attract people from all across the USA also influencing the amouunt of flights.

When examining the relationship between GDP per capita and the volume of air travel, it is important to note that while a higher GDP per capita might generally correlate with increased air travel due to greater economic activity and prosperity, this may not be the sole determining factor. The necessity of air travel in Alaska is driven more by geographical demands than by economic strength as the tourist industry might have a big impact on Florida's flight movements. Furthermore the differences between state's GDP per capita might be to small to have a significant impact. 

Therefore, to accurately assess the influence of GDP per capita on air travel, or vice versa, it is imperative to consider the geographical context and specific characteristics of each region. This nuanced approach ensures a more comprehensive understanding of the factors influencing air travel patterns across different states.

### GDP per capita - International Flights

 In the next part of this research the correlation between a state's GDP per capita and the number of international flight is tested from all the states.

Since the international data flights are listed per cities and not per states, the city are manually mapped to indicate their corresponding states as shown below.

In [31]:

# 1. Count flights for each origin state and year-quarter
origin_intl_flight_counts = merged_df_international.groupby(['ORIGIN_STATE', 'YEAR', 'QUARTER']).size().reset_index(name='Number of Flights')

# Assuming df is your flight data DataFrame and is already loaded
# Grouping the data by "ORIGIN_STATE" and counting the number of flights

# Ensure 'Year' and 'Quarter' columns are in string format
origin_intl_flight_counts['YEAR'] = origin_intl_flight_counts['YEAR'].astype(str)
origin_intl_flight_counts['QUARTER'] = origin_intl_flight_counts['QUARTER'].astype(str)
origin_intl_flight_counts['Year_Quarter'] = origin_intl_flight_counts['YEAR'] + ' Q' + origin_intl_flight_counts['QUARTER']

# Grouping the data by state and year-quarter
state_flight_counts = (
    origin_intl_flight_counts[origin_intl_flight_counts['ORIGIN_STATE'].notnull()]
    .groupby(['ORIGIN_STATE', 'Year_Quarter'])
    .size()
    .reset_index(name='Number of Flights')
)

# Create a DataFrame for state information
state_info = (
    pd.DataFrame(state_coords.items(), columns=['State', 'Coords'])
    .assign(**{'lat': lambda df: df['Coords'].str[0], 'lon': lambda df: df['Coords'].str[1]})
    .drop(columns='Coords')
)

# Merge flight counts with state information
state_intl_flight_counts = pd.merge(origin_intl_flight_counts, state_info, left_on='ORIGIN_STATE', right_on='State')

# Creating the bubble map
fig = px.scatter_geo(
    state_intl_flight_counts,
    lon               = 'lon',
    lat               = 'lat',
    size              = 'Number of Flights',
    color             = 'Number of Flights',
    hover_name        = 'State',
    hover_data        = {'State': False, 'Number of Flights': True},
    color_continuous_scale = 'Viridis',
    size_max          = 50,
    locationmode      = 'USA-states',
    title             = 'Number of International Flights Originating from Each State Over Time',
    scope             = 'usa',
    template          = 'plotly',
    animation_frame   = 'Year_Quarter'
)

# Layout Updates
fig.update_geos(
    landcolor          = 'rgb(210, 180, 140)',  # Light brown color for land
    showocean          = True, oceancolor = 'rgb(135, 206, 250)',  # Light blue color for ocean
    showlakes          = True, lakecolor  = 'rgb(173, 216, 230)',  # Lighter blue color for lakes
    showland           = True,
    showsubunits       = True, subunitcolor = 'rgb(255, 255, 255)',
    showcountries      = False,
    showcoastlines     = True, coastlinecolor = 'rgb(255, 0, 0)',  # Red color for coastline
    showrivers         = True, rivercolor = 'rgb(70, 130, 180)'    # Steel blue color for rivers
)



fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0},
                  coloraxis_colorbar=dict(title='Number of Flights'))

fig.show()


The bubble map above illustrates that Florida, California, Texas, and New York are among the leading states in terms of the number of international flights departing from their airports. Here are Here are several factors contributing to this trend:

* Florida: Home to major tourist destinations and several large international airports, including Miami International Airport, which is a major gateway between the U.S. and Latin America.

* California: Houses major international airports like Los Angeles International Airport (LAX) and San Francisco International Airport (SFO), serving as significant entry points on the West Coast.

* Texas: Has major international airports like Dallas/Fort Worth International Airport (DFW) and George Bush Intercontinental Airport in Houston, serving as important hubs for flights to/from Latin America and other parts of the world.

* New York: New York City atracts millions of international tourists each year with mainly John F. Kennedy International Airport (JFK) serving as one of the main international gateways to the U.S., especially for transatlantic flights.


The significant decrease in the number of flights between 2019 and 2020 for all the states might have been related to the travel restrictions and lockdown during the Covid 19 Pandemic

Now, the correlation between the number of flight departure per state and its GDP per capita is calculated below

In [36]:

# 1a. Count flights for each origin state and year-quarter
origin_intl_flight_counts = merged_df_international.groupby(['ORIGIN_STATE', 'YEAR', 'QUARTER']).size().reset_index(name='Flight_Count')

# 1b. Count flights for each destination state and year-quarter
dest_intl_flight_counts = merged_df_international.groupby(['DEST_STATE', 'YEAR', 'QUARTER']).size().reset_index(name='Flight_Count')

# Rename columns for merge
origin_intl_flight_counts = origin_intl_flight_counts.rename(columns={'ORIGIN_STATE': 'State'})
dest_intl_flight_counts = dest_intl_flight_counts.rename(columns={'DEST_STATE': 'State'})

# 2. Reshape GDP dataframe to long format
gdp_melted_intl = df_GDP_per_capita.melt(id_vars=['GeoName'], value_vars=df_GDP_per_capita.columns[4:], var_name='Year:Quarter', value_name='GDP per capita')

# 3. add column with the state abbreviation
# Ensure that the 'GeoName' column exists in the DataFrame you're working with
gdp_melted_intl['State'] = gdp_melted_intl['GeoName'].apply(state_name_to_abbreviation)

# Split 'Year:Quarter' column into separate 'YEAR' and 'QUARTER' columns
gdp_melted_intl['YEAR'] = gdp_melted_intl['Year:Quarter'].str.split(':', expand=True)[0].astype(int)
gdp_melted_intl['QUARTER'] = gdp_melted_intl['Year:Quarter'].str.extract('Q(\d)')[0].astype(int)

# Convert GDP to numeric
gdp_melted_intl['GDP per capita'] = pd.to_numeric(gdp_melted_intl['GDP per capita'], errors='coerce')

# Drop 'Year:Quarter' after verifying that 'YEAR' and 'QUARTER' are correct
gdp_melted_intl.drop('Year:Quarter', axis=1, inplace=True)

# 4. Merge flight_counts with gdp_melted for both origin and destination states
origin_merged_intl = pd.merge(origin_intl_flight_counts, gdp_melted_intl, on=['State', 'YEAR', 'QUARTER'], how='inner')
dest_merged_intl = pd.merge(dest_intl_flight_counts, gdp_melted_intl, on=['State', 'YEAR', 'QUARTER'], how='inner')

# 5. Compute correlation for each state
origin_correlations = origin_merged_intl.groupby('State').apply(lambda group: group['Flight_Count'].corr(group['GDP per capita']))
dest_correlations = dest_merged_intl.groupby('State').apply(lambda group: group['Flight_Count'].corr(group['GDP per capita']))

# Display the correlations
print("Origin State Correlations:")
print(origin_correlations)
print("\nDestination State Correlations:")
print(dest_correlations)


Origin State Correlations:
State
AK   -0.317891
AL    0.034373
AR    0.061036
AZ    0.045152
CA    0.848374
CO    0.527946
CT    0.414927
DE    0.511673
FL    0.332229
GA    0.156202
HI    0.454522
IA    0.048268
ID   -0.015349
IL    0.852572
IN    0.313827
KS    0.408044
KY    0.527739
LA   -0.211703
MA    0.603377
MD    0.662707
ME   -0.178178
MI    0.050681
MN    0.226974
MO    0.243504
MS    0.068893
MT    0.042898
NC   -0.106022
ND   -0.048090
NE    0.090408
NH    0.058672
NJ    0.526042
NM    0.173327
NV    0.315881
NY    0.401666
OH    0.656764
OK   -0.282577
OR    0.619215
PA   -0.239771
RI    0.351273
SC    0.423334
SD    0.627209
TN    0.369978
TX    0.516586
UT    0.415300
VA    0.218306
VT    0.115793
WA    0.267283
WI   -0.126139
WV   -0.120710
WY   -0.203777
dtype: float64

Destination State Correlations:
State
AK   -0.647712
AL    0.062141
AR   -0.014526
AZ    0.044038
CA    0.845235
CO    0.533999
CT    0.263986
DE    0.479116
FL    0.337042
GA    0.186881
HI    0.50275

In [42]:
import plotly.express as px
import pandas as pd

# Correctly renaming the correlation columns after resetting the index
origin_correlations = origin_correlations.reset_index().rename(columns={0: 'Correlation_Origin'})
dest_correlations = dest_correlations.reset_index().rename(columns={0: 'Correlation_Destination'})

# Merging the two dataframes on 'State' with the correct column names
combined_correlations = pd.merge(origin_correlations, dest_correlations, on='State')

# Melting the combined DataFrame to have 'Type' and 'Correlation' columns suitable for Plotly
melted_correlations = combined_correlations.melt(id_vars='State', value_vars=['Correlation_Origin', 'Correlation_Destination'],
                                                       var_name='Type', value_name='Correlation')

# Creating the bar plot with Plotly Express
fig = px.bar(
    melted_correlations,
    x='State',
    y='Correlation',
    color='Type',
    barmode='group',
    title='Correlation between International Flights and GDP per Capita per State'
)

# Display the figure in your local Jupyter notebook or Python script
fig.show()


* States with positive correlation coefficients indicate that as GDP increases, the number of international flights also tends to increase. This could suggest that in these states, economic growth or size may be linked to greater international air traffic.
* Conversely, states with negative correlation coefficients indicate an inverse relationship; as GDP increases, the number of international flights tends to decrease. This might be due to a variety of factors, such as alternative modes of international transport being more prevalent, or that the international flights are not directly tied to the economic activities measured by GDP in these states.
* States that have bars close to zero suggest little to no linear relationship between GDP and the number of international flights. For these states, factors other than GDP might be more significant in influencing the number of international flights.

Generally, the chart could be interpreted to reflect the complexity of economic activity and its relationship to international connectivity. States that serve as major transportation hubs or have substantial international business and tourism industries might show a stronger positive correlation. Futhermore, it is needed to have an understanding of the tactics used in the aircraft industry like a hub and spoke model. These desicions have a big influence on where international flights depart and arrive possible more than the state's GDP per capita.

## Sub question 2: 
- ## Did the impact of the COVID-19 pandemic on air traffic movements in the United States vary significantly among states with different levels of GDP per capita?

The COVID-19 Pandemic started in the end of 2019 and the main effects went on till at least half way in 2021. To get a complete as possible visualization the flight data from 2019-2021 is used. This give the opportunity to show the abrupt impact from when it started and the slow recovery in the end of the Pandemic.

## Domestic Flight

In [26]:
state_dom_flight_counts

NameError: name 'state_dom_flight_counts' is not defined

In [46]:
import plotly.express as px

# Convert the 'YEAR' column to integers
state_dom_flight_counts['YEAR'] = state_dom_flight_counts['YEAR'].astype(int)

# Now filter the DataFrame for the years 2019 to 2021
filtered_df = state_dom_flight_counts[(state_dom_flight_counts['YEAR'] >= 2019) & (state_dom_flight_counts['YEAR'] <= 2021)]

# Filter the DataFrame for the years 2019 to 2021
filtered_df = state_dom_flight_counts[(state_dom_flight_counts['YEAR'] >= 2019) & (state_dom_flight_counts['YEAR'] <= 2021)]

# Create an interactive line chart
fig = px.line(
    filtered_df,
    x='Year_Quarter',
    y='Number of Flights',
    color='State',
    title='Historical Trend of Flight Counts (2019-2021)'
)

# Update the layout for better readability
fig.update_layout(
    xaxis_title='Year and Quarter',
    yaxis_title='Number of Flights',
    legend_title='State',
    hovermode='closest'
)

# Enable the user to interact with the plot to see details
fig.update_traces(mode='lines+markers')

# Show the figure
fig.show()


The following conclusions have been drawn regarding the impact of COVID-19 on domestic flights in the USA:

* Significant Reduction in Flights in 2020: There is a notable decline in the number of flights across almost all states starting from 2020 Q1 or Q2, which aligns with the onset of COVID-19 and the implementation of travel restrictions and lockdowns.

* Varied Impact by State: The impact of COVID-19 seems to vary by state. Some states show a more pronounced decline in flight numbers than others, which could be due to differences in state policies, the severity of COVID-19 cases, or economic factors.

* Recovery Phase: There appears to be a recovery phase starting in late 2020 or early 2021, where the number of flights begins to increase again. This could be attributed to the easing of travel restrictions, the rollout of vaccines, or improved public health measures.

* Not Back to Pre-Pandemic Levels: By the end of 2021, while there is a recovery, the flight counts for most states do not appear to have returned to pre-pandemic levels, indicating a lasting impact of the pandemic on domestic air travel.

The impact of COVID-19 on domestic flight numbers is evident across all states, suggesting a reduction in flights that is independent of each state's GDP per capita levels.

## International Flights

In [47]:
state_intl_flight_counts

,ORIGIN_STATE,YEAR,QUARTER,Number of Flights,Year_Quarter,State,lat,lon
0,AK,2005,1,266,2005 Q1,AK,61.370716,-152.404419
1,AK,2005,2,269,2005 Q2,AK,61.370716,-152.404419
2,AK,2005,3,300,2005 Q3,AK,61.370716,-152.404419
3,AK,2005,4,265,2005 Q4,AK,61.370716,-152.404419
4,AK,2006,1,248,2006 Q1,AK,61.370716,-152.404419
...,...,...,...,...,...,...,...,...
3348,WY,2021,3,2,2021 Q3,WY,42.755966,-107.302490
3349,WY,2022,1,3,2022 Q1,WY,42.755966,-107.302490
3350,WY,2022,2,1,2022 Q2,WY,42.755966,-107.302490
3351,WY,2022,3,4,2022 Q3,WY,42.755966,-107.302490


In [50]:
import plotly.express as px

# Convert the 'YEAR' column to integers
state_intl_flight_counts['YEAR'] = state_intl_flight_counts['YEAR'].astype(int)



# Filter the DataFrame for the years 2019 to 2021
filtered_intl = state_intl_flight_counts[(state_intl_flight_counts['YEAR'] >= 2019) & (state_intl_flight_counts['YEAR'] <= 2021)]

# Create an interactive line chart
fig = px.line(
    filtered_intl,
    x='Year_Quarter',
    y='Number of Flights',
    color='State',
    title='Historical Trend of USA International Flight Counts (2019-2021)'
)

# Update the layout for better readability
fig.update_layout(
    xaxis_title='Year and Quarter',
    yaxis_title='Number of Flights',
    legend_title='State',
    hovermode='closest'
)

# Enable the user to interact with the plot to see details
fig.update_traces(mode='lines+markers')

# Show the figure
fig.show()


USA international flights appear to have been more significantly impacted by COVID-19 than domestic flights, as evidenced by the greater decrease in flight numbers on the graph above.

The following conclusions have been drawn about the impact of COVID-19 on international flights:

* Sharp Decline in Early 2020: There is a significant drop in the number of international flights across various states starting from early 2020, which coincides with the global spread of COVID-19 and the implementation of international travel restrictions.

* Varied Impact by State: Similar to domestic flights, the impact on international flights varies by state, which could be due to the varying roles states play in international air travel, with some states having more international airports or serving as major hubs for international flights.

* Gradual Recovery: The graph shows a gradual increase in international flight counts starting around mid-2020 or early 2021, suggesting a slow recovery as travel restrictions began to lift and as the international community responded to the pandemic.

* Incomplete Recovery by End of 2021: Despite the upward trend in recovery, by the end of 2021, the number of international flights has not returned to pre-pandemic levels, indicating a prolonged impact of the pandemic on international travel.

* Potential Resurgence in Late 2021: There is an observable increase in flights towards the end of 2021 in some states, which may indicate a resurgence in international travel, possibly due to vaccine rollouts and easing of travel bans.


Alaska (AK) presents an anomaly in the data, as it exhibits an increase in the number of international flights in early 2020, contrasting with the decrease observed in other states. This deviation may be attributed to Alaska's unique geographical position; being non-contiguous and separated by sea from the continental United States, Alaska's transportation dynamics are inherently different. Factors such as its reliance on air travel for connectivity to both domestic and international destinations, and possibly less stringent travel restrictions during the initial phase of the pandemic, could have contributed to this trend. Additionally, Alaska serves as a strategic stopover point for some international flights, which might have affected its flight count despite global travel disruptions.

## Sub question 3:
- ## Were there pivotal moments or significant turning points in the relationship between state GDP per capita and air traffic movements in the United States, and what factors contributed to these shifts?

To conclude whether there were pivotal moments or significant turning points in the relationship between state GDP per capita and air traffic movements in the United States, the graphs of sub questions 1 and sub question 2 are analysed.

As analysed in sub-question 2, the Covid-19 period had a major impact on air traffic movements in the United States.
To see if there were any other major changes, the graph on the historical trend of U.S. State GDP per Capita (2005-2022) from sub question 1 is analysed. In the graph no other pivotal moment can be seen, besides the big drop in 2019-2020, attributable to the Covid period. The graph in subquestion 1 on the number of domestic and international flights from each state over time does not show any other pivotal moment either.

In conclusion, as expected, Covid-19 is the only pivotal moment that affected air traffic movements and the state GDP per capita.